In [1]:
from parmed import gromacs

In [2]:
import parmed as pmd

In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
top = gromacs.GromacsTopologyFile('exmaple.top', parametrize=False, xyz='exmaple.gro')

In [7]:
elem_dict = {
    "H":1, 
    "C":6,
    "N":7,
    "O":8,
    "F":9,
    "P":15,
    "S":16,
    "CL":17,
    "BR":35,
    "I":53
}

def get_an(a):
    string = a.name.upper()
    start = string[:2]
    first = string[0]
    
    if start.upper == "CL":
        elem = 17
    elif start.upper == "BR":
        elem = 35
    else:
        elem = elem_dict[first] 
    
    print(f"Repplacing {a.name}, atomic_no: ({a.atomic_number}) with {elem}")
    return elem 

In [8]:
ff_old = """
 ffio_ff { 
  :::
  ffio_vdwtypes[0] { 
   :::
   :::
  } 
  ffio_sites[0] { 
   :::
   :::
  } 
  ffio_bonds[0] { 
   :::
   :::
  } 
  ffio_angles[0] { 
   :::
   :::
  } 
  ffio_dihedrals[0] { 
   :::
   :::
  } 
  ffio_exclusions[0] { 
   :::
   :::
  } 
  ffio_pairs[0] { 
   :::
   :::
  } 
  ffio_restraints[0] { 
   :::
   :::
  } 
  ffio_virtuals[0] { 
   :::
   :::
  } 
  ffio_pseudo[0] { 
   :::
   :::
  } 
  ffio_constraints[0] { 
   :::
   :::
  } 
  ffio_posre_fbhw[0] { 
   :::
   :::
  } 
  ffio_angle_fbhw[0] { 
   :::
   :::
  } 
  ffio_improper_fbhw[0] { 
   :::
   :::
  } 
  ffio_stretch_fbhw[0] { 
   :::
   :::
  } 
  ffio_vdwtypes_combined[0] { 
   :::
   :::
  } 
 } 
} 

"""

In [9]:
def write_section(top, traj_file, sys, f, tmp_ffio):
    coords = top.coordinates
    resids = top.residues
    atoms  = top.atoms
    vectors = top.box_vectors
    bonds = top.bonds
    
    # not tested for None
    if vectors is not None:
        boxx, boxy, boxz = vectors[0][0]._value, vectors[1][1]._value, vectors[2][2]._value
    else:
        boxx, boxy, boxz = 8,8,8
        
    if traj_file is not None:
        last_segment = "  r_chorus_box_cz\n  s_chorus_trajectory_file\n"
        last_value = f"  {boxz}\n  {traj_file}\n"
    else:
        last_segment = "  r_chorus_box_cz\n"
        last_value = f"  {boxz}\n"
            
    f.write("f_m_ct {\n"
                +"  s_m_title\n"
                +"  r_chorus_box_ax\n"
                +"  r_chorus_box_ay\n"
                +"  r_chorus_box_az\n"
                +"  r_chorus_box_bx\n"
                +"  r_chorus_box_by\n"
                +"  r_chorus_box_bz\n"
                +"  r_chorus_box_cx\n"
                +"  r_chorus_box_cy\n"
                +last_segment
                +"  s_ffio_ct_type\n"
                +"  :::\n"
                +"  \"Converted file\"\n"
                +f"  {boxx}\n"
                +"  0\n"
                +"  0\n"
                +"  0\n"
                +f"  {boxy}\n"
                +"  0\n"
                +"  0\n"
                +"  0\n"
                +last_value
                +f"  {sys}\n"
                +f"  m_atom[{len(atoms)}] {{\n"
                +"    # First column is atom index #\n"
                +"    i_m_mmod_type\n"
                +"    r_m_x_coord\n"
                +"    r_m_y_coord\n"
                +"    r_m_z_coord\n"
                +"    i_m_residue_number\n"
                +"    i_m_color\n"
                +"    s_m_pdb_residue_name\n"
                +"    i_m_atomic_number\n"
                +"    s_m_atom_name\n"
                +"    s_m_pdb_atom_name\n"
                +"    i_m_formal_charge\n"
                #+"    r_ffio_x_vel\n"
                #+"    r_ffio_y_vel\n"
                #+"    r_ffio_z_vel\n"
                +"    :::\n")
        
    for i, (c, a) in enumerate(zip(coords, atoms), start=1):
        res = a.residue
        if a.mass == 0:
            an = get_an(a)
        else:
            an = a.atomic_number
            
        if res.name == "SOL":
            resn = '"SPC "'
        else:
            resn = res.name
            
        f.write(f"    {i} 1 {c[0]} {c[1]} {c[2]} {res.number} 1 {resn} {an} {a.name} {a.name} {a.charge}\n")
        #print(a.element, a.element_name, a.name, a.mass)

        #bonds
    f.write("    :::\n"
                +"  }\n"
                +f"  m_bond[{len(bonds)}] {{\n"
                +"    i_m_from\n"
                +"    i_m_to\n"
                +"    i_m_order\n"
                +"    :::\n")
        
    for i, b in enumerate(bonds, start=1):
        a1 = b.atom1
        a2 = b.atom2
        f.write(f"    {i} {a1.idx+1} {a2.idx+1} 1\n")
    f.write("    :::\n"
            +"  }\n")
            
        #ffio_ff
    if tmp_ffio:
        f.write(ff_old)
    else:
        f.write("  ffio_ff {\n"
                    +"    :::\n"
                    +f"    ffio_sites[{len(atoms)}] {{\n"
                    +"      s_ffio_type\n"
                    +"      r_ffio_charge\n"
                    +"      r_ffio_mass\n"
                    +"      :::\n")
            
        for i, a in enumerate(atoms, start=1):
            f.write(f"      {i} atom {a.charge} {a.mass}\n")
        f.write("      :::\n    }")
        f.write("\n  }\n}\n\n")

In [11]:
def write_cms(top, filename, traj_file=None, pdb_numbers=None):

    with open(filename, 'w') as f:
        #s_m_m2io_version
        f.write("{\n"
                +"  s_m_m2io_version\n"
                +"  :::\n"
                +"  2.0.0\n"
                +"}\n")
    
        write_section(top=top, traj_file=traj_file, sys="full_system", f=f, tmp_ffio=False)
        write_section(top=top, traj_file=None, sys="solute", f=f, tmp_ffio=True)

In [12]:
write_cms(top, filename='exmaple.cms', traj_file='exmaple.xtc')

Repplacing C7, atomic_no: (0) with 6
Repplacing C6, atomic_no: (0) with 6
Repplacing N3, atomic_no: (0) with 7
Repplacing C10, atomic_no: (0) with 6
Repplacing C9, atomic_no: (0) with 6
Repplacing C8, atomic_no: (0) with 6
Repplacing N2, atomic_no: (0) with 7
Repplacing H3, atomic_no: (0) with 1
Repplacing H4, atomic_no: (0) with 1
Repplacing C7, atomic_no: (0) with 6
Repplacing C6, atomic_no: (0) with 6
Repplacing N3, atomic_no: (0) with 7
Repplacing C10, atomic_no: (0) with 6
Repplacing C9, atomic_no: (0) with 6
Repplacing C8, atomic_no: (0) with 6
Repplacing N2, atomic_no: (0) with 7
Repplacing H3, atomic_no: (0) with 1
Repplacing H4, atomic_no: (0) with 1
Repplacing C7, atomic_no: (0) with 6
Repplacing C6, atomic_no: (0) with 6
Repplacing N3, atomic_no: (0) with 7
Repplacing C10, atomic_no: (0) with 6
Repplacing C9, atomic_no: (0) with 6
Repplacing C8, atomic_no: (0) with 6
Repplacing N2, atomic_no: (0) with 7
Repplacing H3, atomic_no: (0) with 1
Repplacing H4, atomic_no: (0) with 